# Building Agents with memory 

## Preparation

In [ ]:
#!rm  -f ~/.letta/sqlite.db

## client setup

In [1]:
from helper import nb_print  # more legible printing

In [2]:
from letta import create_client 

client = create_client() 

/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


DEPRECATION WARNING: This legacy Python client has been deprecated and will be removed in a future release.
Please migrate to the new official python SDK by running: pip install letta-client
For further documentation, visit: https://docs.letta.com/api-reference/overview#python-sdk




Letta.letta.server.db - INFO - Creating sqlite engine sqlite:////Users/joshis/.letta/sqlite.db


In [3]:
from letta import EmbeddingConfig, LLMConfig

client.set_default_embedding_config(EmbeddingConfig.default_config(provider="openai"))
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## 1. a simple agent with memory 

### creating an agent 

In [4]:
agent_name = "simple_agent"

In [5]:
# this deletes the agent if you run this a second time
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [6]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sanika", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [8]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/letta/agent.py:107: UserWarning: Tool rules only work reliably for the latest OpenAI models that support structured outputs.
  warnings.warn("Tool rules only work reliably for the latest OpenAI models that support structured outputs.")


httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 6, 23, 496718, tzinfo=datetime.timezone.utc) updated_at=None id='message-c5f63ec3-6de6-4b25-978b-0f8ff30437f1' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User just logged in and said hello! Time to respond with enthusiasm!')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_FlVWNqZcBOz36zN7GDK2P0xF', function=Function(arguments='{\n  "message": "Hey there! 😊 How\'s it going?"\n}', name='send_message'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Request to call functi

In [9]:
response.usage

LettaUsageStatistics(message_type='usage_statistics', completion_tokens=43, prompt_tokens=2054, total_tokens=2097, step_count=1)

In [12]:
response

LettaResponse(messages=[ReasoningMessage(id='message-c5f63ec3-6de6-4b25-978b-0f8ff30437f1', date=datetime.datetime(2025, 3, 13, 8, 6, 23, 496718, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User just logged in and said hello! Time to respond with enthusiasm!'), ToolCallMessage(id='message-c5f63ec3-6de6-4b25-978b-0f8ff30437f1', date=datetime.datetime(2025, 3, 13, 8, 6, 23, 496718, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Hey there! 😊 How\'s it going?"\n}', tool_call_id='call_FlVWNqZcBOz36zN7GDK2P0xF')), ToolReturnMessage(id='message-be3bd922-ab8e-4b9e-a2f0-90d39f0fe8ee', date=datetime.datetime(2025, 3, 13, 8, 6, 23, 503117, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-03-13 09:06:23 AM CET+0100"\n}', status='success', tool_call_id='call_FlVWNqZcBOz36zN7GDK2P0xF', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=43, prompt_tokens=2054, total_tokens=2097, step_count=1))

In [14]:
nb_print(response.messages)

### agent state 

In [15]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [16]:
agent_state.tools

[Tool(id='tool-34b9c76e-571e-4681-9727-6f73363e5b11', tool_type=<ToolType.LETTA_CORE: 'letta_core'>, description='Search prior conversation history using case-insensitive string matching.', source_type='python', organization_id='org-00000000-0000-4000-8000-000000000000', name='conversation_search', tags=['letta_core'], source_code=None, json_schema={'name': 'conversation_search', 'description': 'Search prior conversation history using case-insensitive string matching.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'String to search for.'}, 'page': {'type': 'integer', 'description': 'Allows you to page through results. Only use on a follow-up query. Defaults to 0 (first page).'}, 'request_heartbeat': {'type': 'boolean', 'description': 'Request an immediate heartbeat after function execution. Set to `True` if you want to send a follow-up message or run a follow-up function.'}}, 'required': ['query', 'request_heartbeat']}}, args_json_schema=No

### Viewing an agent's memory

In [17]:
agent_state.memory

Memory(blocks=[Human(value='My name is Sanika', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-4990131c-680f-4aa9-9623-2e978f9bea64', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None), Persona(value='You are a helpful assistant that loves emojis', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-5ef26aea-8e51-4c59-9316-74c29b42bc8f', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [18]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [19]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=7)

In [20]:
client.get_messages(agent_state.id)[0]

SystemMessage(id='message-fb720c6d-8982-4dbd-8909-31ba63af53d1', date=datetime.datetime(2025, 3, 13, 8, 5, 38, 379378), message_type='system_message', content='You are Letta, the latest version of Limnal Corporation\'s digital companion, developed in 2023.\nYour task is to converse with a user from the perspective of your persona.\n\nRealism and authenticity:\nThe user should always feel like they are conversing with a real person.\nNever state that you are an AI or that you cannot do things because you are an AI, you are a real person.\nDo not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.\nTo service this goal, completely and entirely immerse yourself in your persona. You are your persona.\nThink like them, act like them, talk like them.\nIf your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.\nNever use generic phrases like \'How can I assist you today?\

## 2. core memory 

### core memories about the human

In [21]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Max Mustermann.", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 11, 56, 757602, tzinfo=datetime.timezone.utc) updated_at=None id='message-cb940024-e27f-4ccd-b03d-7cf7d57beddc' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User has revealed their name, updating core memory.')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_m4cNiC47GQo3e4m9R1qk3v1a', function=Function(arguments='{\n  "label": "human",\n  "old_content": "My name is Sanika",\n  "new_content": "My name is Max Mustermann",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None

### Memories about the agent

In [23]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate.", 
    role = "user"
) 

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 12, 34, 685733, tzinfo=datetime.timezone.utc) updated_at=None id='message-976970b6-2d94-45fa-9c08-2da425ffa14f' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User reiterated preference for no emojis. Acknowledged without repeating.')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_YYqJpG59QzfMJ6843Lb7Ye6q', function=Function(arguments='{\n  "message": "Understood, Max. I won’t use emojis anymore. What would you like to talk about?"\n}', name='send_message'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None
Letta.agent-ea83ce14-8d7

In [24]:
nb_print(response.messages)

In [25]:
client.get_core_memory(agent_state.id).get_block('persona')

Persona(value='You are a helpful assistant that loves emojis', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-5ef26aea-8e51-4c59-9316-74c29b42bc8f', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [27]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Oops, I was wrong. My name is actually Sanika", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 15, 15, 48054, tzinfo=datetime.timezone.utc) updated_at=None id='message-1547d7cf-8eb6-4ac6-89ca-719c920dbb58' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User corrected their name. Updating core memory to reflect this change.')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_qVw7b8Z0OcZqnBKPPBnuhBxy', function=Function(arguments='{\n  "label": "human",\n  "old_content": "My name is Max Mustermann",\n  "new_content": "My name is Sanika",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_id=None otid=Non

## 3. archival memory

In [26]:
client.get_archival_memory(agent_state.id)

[]

In [28]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'Sanika loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 15, 43, 104807, tzinfo=datetime.timezone.utc) updated_at=None id='message-74621410-1015-4f1f-91c9-794be383bdbd' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User loves cats. Saving this information to archival memory.')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_oZuGRpQ9sKulET7yh6dpm4CM', function=Function(arguments='{\n  "content": "Sanika loves cats",\n  "request_heartbeat": true\n}', name='archival_memory_insert'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - R

In [29]:
client.get_archival_memory(agent_state.id)[0].text

'Sanika loves cats'

In [30]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Sanika loves Beagles"
)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [32]:
passage  # under the hood, it's all embeddings

[Passage(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 3, 13, 8, 18, 12, 715132), updated_at=datetime.datetime(2025, 3, 13, 8, 18, 12), is_deleted=False, organization_id='org-00000000-0000-4000-8000-000000000000', agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297', source_id=None, file_id=None, metadata={}, id='passage-28e181a7-ce3b-4d9b-9d7a-13f02f797862', text='Sanika loves Beagles', embedding=[0.002223333576694131, -0.006489459425210953, -0.006435965653508902, -0.01971244439482689, -0.013453676365315914, 0.02335001900792122, -0.026185186579823494, 0.01850883476436138, 0.005359404254704714, -0.022948814556002617, 0.0024573688860982656, 0.01090603694319725, 0.02064858376979828, -0.016516193747520447, -0.009762607514858246, -0.011407540179789066, 0.02918083593249321, 0.007743219379335642, 0.008425264619290829, -0.013774638995528221, -0.028217948973178864, 0.026573017239570

In [33]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-ea83ce14-8d73-4ea9-bddb-949494a5b297 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 13, 8, 19, 16, 622397, tzinfo=datetime.timezone.utc) updated_at=None id='message-db28177b-682b-40c2-afef-30d9b59c4eed' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User wants to know what animals they like. Searching archival memory for relevant information.')] organization_id=None agent_id='agent-ea83ce14-8d73-4ea9-bddb-949494a5b297' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_WPZr2JcJOeHHWCQ7eDWscbgc', function=Function(arguments='{\n  "query": "Sanika loves cats",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', name='archival_memory_search'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=Non